In [59]:
!pip install -q noisereduce
!pip install -q git+https://github.com/TenzinGayche/num2tib.git

In [122]:
import librosa
import numpy as np
import torch
import pyewts
import noisereduce as nr
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from num2tib.core import convert
from num2tib.core import convert2text
import re
from pathlib import Path
from IPython.display import Audio, display

from phonetic import text2phon

In [123]:
Path.cwd()

PosixPath('/root/speech_t5_tts/tests')

In [124]:
converter = pyewts.pyewts()
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model_checkpoint = ""
model = SpeechT5ForTextToSpeech.from_pretrained(model_checkpoint)
model.to('cuda')
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

OSError: Incorrect path_or_model_id: '/data/volume/models/exp3-poc/TTS_st5_phono_20k/checkpoint-9390/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
def replace_numbers_with_convert(sentence, wylie=True):
    pattern = r'\d+(\.\d+)?'
    def replace(match):
        return convert(match.group(), wylie)
        
    result = re.sub(pattern, replace, sentence)

    return result

def num2letter(sentence):
    tibetan_nums = "༠༡༢༣༤༥༦༧༨༩"
    for i, n in enumerate(tibetan_nums):
        sentence = sentence.replace(n, str(i))
    result = replace_numbers_with_convert(sentence, wylie=False)
    return result

replacements = [
    ('è', 'e'),  # Similar to "e" in "bed"
    ('ö', 'oe'), # Similar to "o" in "bird" or "u" in "burn" (British accent)
    ('ü', 'u'),  # Similar to "u" in "lune" or "y" in "myth" with rounded lips
    ('ɪ', 'i'),  # Similar to "i" in "bit"
    ('ʂ', 'sh'), # Similar to "sh" in "sheep" with retroflexion
    ('ḍ', 'd'),  # Retroflex "d" similar to hard "d"
    ('ṅ', 'ng'), # Similar to "ng" in "sing"
    ('ṭ', 't'),  # Retroflex "t" similar to hard "t"
]

def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [125]:
num2letter("ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །")

'ཀུན་གླེང་གསར་འགྱུར། གཉིས་སྟོང་ཉི་ཤུ་རྩ་གསུམ་་ལོའི་ཟླ་དགུ་ ཚེས་ཉི་ཤུ་རྩ་བདུན །'

In [119]:
def predict(text):
    if len(text.strip()) == 0:
        return (16000, np.zeros(0).astype(np.int16))

    text = num2letter(text)
    text = text2phon(text)
    text = cleanup_text(text)
    print(text)
    inputs = processor(text=text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    input_ids = input_ids[..., :model.config.max_text_positions]
    speaker_embedding = np.load("female_2.npy")
    speaker_embedding = torch.tensor(speaker_embedding)
    speech = model.generate_speech(input_ids.to('cuda'), speaker_embedding.to('cuda'), vocoder=vocoder.to('cuda'))
    speech = nr.reduce_noise(y=speech.to('cpu'), sr=16000)
    return (16000, speech)

In [120]:
examples = [
    ["ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།", "Lhasa(female)"],
    ["སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།", "Lhasa(female)"],
    ["དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག། ", "Lhasa(female)"],
    ["ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་། ", "Lhasa(female)"],
    ["ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།", "Lhasa(female)"],
    ["ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །", "Lhasa(female)"],
]

In [121]:
for text, _ in examples:
    rate, speech = predict(text)
    print(text)
    display(Audio(speech, rate=rate))

couldn't understand syllable རཱབྷིསས
couldn't understand syllable རཱབྷིསས
ta  té  tso  di  che  goe  re  na  tsa  ma  yong  ngoen  la  ko  rang  la  yang  kyi  ko  rang  ko  la  yang  kap  gyak  ya  yoe  re  un  ngoen  gok  tang    kap  gyak  ya  yoe  re  ta
ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།


top  chen  gyel  kap  nyi  shui  len  tsok  tok  la  gya  nak  shung  ki  poe  nang  rik  shung  tsa  tor  kyi  si  chu  kak  par  ri  tok  ne  le  gul  pel  pai  kor  jam  yang  gya  tso  lak  kyi  nyen  doen  nang  ki  re
སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།


couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ
ngoe  ne  lap  goe    ta  mi  ul  po  té  tso  la  ka  ré  lap  goe  re  jin  pa  tang  ya  yoe    un  té  da  ki  le  gul  té  dai  mang  po  tsam  ki  duk  pa  té  tso  yang  nge  tse  le  gel  pai  yak  po  re  ten  ki  duk
དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག། 


kong  nam  pa  ni  chir  tang  ki  kang  sak  chik  ma  yin  par  mi  rap  ne  mi  rap  gyu  pa  zin  pai  nor  pu  yin  shing
ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་། 


a  lé  é  ne  chir  tang  ta  nga  tso  ta  ta  kye  rang  ki  mik  yul  ta  kel  chen  po  tsi  ne
ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།


kun  léng  sar  gyur  nyi  tong  nyi  shu  tsa  sum  loi  da  gu  tse  nyi  shu  tsa  dun
ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །
